# Higgs Boson challenge

# Exploratory data analysis - feature selection


This dataset has been built from official ATLAS full-detector simulation, with "Higgs to tautau" events mixed with different backgrounds. The task is to classify events into "tau tau decay of a Higgs boson" versus "background".


## Dataset characteristics

[Kaggle Challenge Page](https://www.kaggle.com/c/higgs-boson)

The dataset from Kaggle has 800000 events (195.5 MB in total):
- Training set of 250000 events
- Test set of 550000 events

Training set has 30 feature columns, a weight column and a label column.
Test set has 30 feature columns and a label column.


### Feature characteristics

- all variables are floating point, except PRI_jet_num which is integer
- variables prefixed with PRI (for PRImitives) are “raw” quantities about the bunch collision as measured by the detector.
- variables prefixed with DER (for DERived) are quantities computed from the primitive features, which were selected by  the physicists of ATLAS
- it can happen that for some entries some variables are meaningless or cannot be computed; in this case, their value is −999.0, which is outside the normal range of all variables

### Class distribution

The class distribution of the training set is

- b (background) : 164333 events (66%)
- s (Higgs to tau tau):  85667 (34%)

We can see there is a class imbalance.

In [ ]:
# Usual imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.figsize':(9,6)})
plt.rcParams['figure.figsize'] = (9,6)

import IPython.display as ipd
pd.options.display.max_columns = 35

# random seed for reproducibility
np.random.seed(42)

In [ ]:
df = pd.read_csv("data/training.csv",sep=',').astype({'Label': 'category'})
print("Raw training data:")
ipd.display(df.tail(10))

data = df.drop(['EventId', 'Label', 'Weight'], axis=1)
kaggle_weights = df['Weight']
# encode categorical class variable
target = df['Label']
# replace -990.0 values with NaNs
data = data.replace(-999.0, np.NaN)
weight = df['Weight']

In [ ]:
# number of classes
ipd.display(target.value_counts())
ipd.display(data.describe())

## Missing values

-990.0 values represent entries that have no meaning/that could not be calculated. The figure below shows the the percentage of those missing values per column.

We can see than there are a lot of missing values: 72% of rows contain at least one missing value and  of some columns have up to 71% of them. So it is not viable to delete rows or columns, otherwise it will throw away too much data. Moreover the fact that a variable has no meaning or couldn't be calculated might be correlated with the label.

The three possibilities we have to solve the problem of missing data are:
- conserve -990.0 values: the value was chosen to be abnormal and distant from real data points; hence a model could be capable of identifying it and treat this value as missing.
- replace with NaN values and use algorithms that manage missing values (for example XGBOOST)
- impute those value swith the median (more robust than the man), this will smooth the data but may remove the explicative potential of those values.

In [ ]:
# show percentage of nans per column
perc_nans = data.isna().sum() * 100 / len(data)
perc_rows_nans = (len(data) - len(data.dropna()))*100/(len(data))
print("Percentage of rows containing at least one missing value:", perc_rows_nans)
plt.figure(figsize=(9,6))
ax = perc_nans.plot.barh()
ax.set_xlim(0, 100)
ax.set_xticks(range(0, 101, 10))
ax.tick_params(axis='y', labelsize=10)
plt.title("Percentage of Missing Values Per Column\n", fontsize=15)
#plt.savefig("figures/Percentage_of_Missing_Values_Per_Column.eps", bbox_inches = "tight")
plt.show()

ipd.display(perc_nans)

## Histograms

To get a better understanding of the features distributions we plot the histograms for each features. 

Some variables seem to follow well known probability distributions:

- Gaussians (ex. `PRI_tau_eta`, `PRI_lep_eta`)
- Beta distribution with one parameter alpha (U distribution) (`DER_lep_eta_centrality`, `DER_met_phi_centrality`)
- Uniform distributions (ex. `PRI_tau_phi`, `PRI_met_phi`)
- Exponential distributions (ex. `DER_sum_pt`, `PRI_jet_leading_pt`)


In [ ]:
# show histograms of features
data.hist(bins=30, figsize=(27,18))
plt.suptitle("Features Histograms", fontsize=30, y=0.94)
#plt.savefig("figures/Histograms.svg")
plt.show()

## Per-class histograms for primitive features and derived features

Here, the features are separated into primitives (PRI) and derived (DER). For each feature we plot two histograms, one per class, to see any differences in distribution.

Certain features, for ex. `DER_mass_MMC` and `DER_mass_transverse_met_lep` have significative differences in their distributions between classes, they will probably be discriminant.

In [ ]:
# show histograms of primitives features

pri = df.loc[:, df.columns.str.startswith(('PRI', 'Label'))]
pri_columns = pri.drop('Label', axis=1).columns

# show histograms of features
#data.hist(bins=30, figsize=(27,18))
#plt.show()

def sephist(df, col):
    s = df[df['Label'] == 's'][col]
    b = df[df['Label'] == 'b'][col]
    return s, b

plt.figure(figsize=(27, 18),)

for num, column in enumerate(pri_columns):
    plt.subplot(4, 5, num+1)
    sep = sephist(pri, column)
    plt.hist(sep[0], bins=30, density=True, alpha=0.6, label='s')
    plt.hist(sep[1], bins=30, density=True, alpha=0.6, label='b')
    plt.legend(loc='best')
    plt.title(column)

# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.suptitle("Primitive Features Histograms", fontsize=28, y=0.94)
#plt.savefig("figures/Primitive_Features_Histograms.svg")
plt.show()

# show histograms of primitives features

der = df.loc[:, df.columns.str.startswith(('DER', 'Label'))]
der_columns = der.drop('Label', axis=1).columns

# show histograms of features
#data.hist(bins=30, figsize=(27,18))
#plt.show()

def sephist(df, col):
    s = df[df['Label'] == 's'][col]
    b = df[df['Label'] == 'b'][col]
    return s, b

plt.figure(figsize=(21, 13))

for num, column in enumerate(der_columns):
    plt.subplot(4, 5, num+1)
    sep = sephist(der, column)
    plt.hist(sep[0], bins=30, density=True, alpha=0.6, label='s')
    plt.hist(sep[1], bins=30, density=True, alpha=0.6, label='b')
    plt.legend(loc='best')
    plt.title(column)

#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.suptitle("Derived Features Histograms", fontsize=22, y=0.94)
#plt.savefig("figures/Derived_Features_Histograms.svg")
plt.show()


## Correlations matrices

We compute the pairwise correlation of columns, excluding NaN values, with the Pearson's coefficient.  

We can observe that several variables are highly correlated (for example `DER_sum_pt` with `PRI_jet_all_pt` (0.97)). As some variables are derived from others this is expected. There are also high correlations between primitives variables (ex. `PRI_met_sumet` with `PRI_jet_all_pt` (0.88))

In [ ]:
# Correlation matrix
corr = data.corr()
corr.index = data.columns
plt.figure(figsize=(30,30))
sns.heatmap(corr, annot=True, cmap='RdYlGn', fmt='.2f')
plt.title("Correlation Heatmap\n", fontsize=30)
plt.show()

In [ ]:
pri = pri_data = data.loc[:, data.columns.str.startswith('PRI')]

# Correlation matrix
pri_corr = pri_data.corr()
pri_corr.index = pri_data.columns
plt.figure(figsize=(16,16))
sns.heatmap(pri_corr, annot=True, cmap='RdYlGn', fmt='.2f')
plt.title("Correlation Heatmap\n", fontsize=16)
plt.show()

## Analysis of the importance of features via a random forest classifier.

To get a grasp of feature importances for classification, the best method we see is an "a posteriori" selection from the learning of a classifier capable of assigning importance to each of the features.
We train a random forest with untuned hyperparams to get a scores of features importances.

For each decision tree, the importance of a feature importance is computed as the (normalized) total reduction of the Grini criterion brought by that feature. It is also known as the Gini importance. It follows that the importances of features for the random forest are the mean of those importances for each tree.

In [ ]:
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(target)
target_enc = le.transform(target)

#mi = mutual_info_regression(imp_data, target_enc)
#mi /= np.max(mi)

## Random Forest Classifier to visualize feature importances

By fitting classifier like a random forest on the data, we can observe the importance of features for the classification afterwards.

For each decision tree, the importance of a feature importance is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance.
It follows that the importances of features for the random forest are the mean of those importances for each tree. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

y = target.cat.codes
imputer = SimpleImputer(strategy='median')
X = imputer.fit_transform(data.values)

clf = RandomForestClassifier(n_estimators=150, max_depth=10, n_jobs=-1)
clf.fit(X, y)

In [ ]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)[indices]
# Print the feature ranking
print("Feature ranking:")
importances = pd.Series(clf.feature_importances_, index=data.columns)[indices]

# Plot the feature importances of the forest
plt.figure(figsize=(9,6))
plt.title("Feature Importances of Random Forest", fontsize=15)

importances.plot.bar(yerr=std, color='r')
plt.gcf().subplots_adjust(bottom=0.45)

#plt.savefig("figures/Feature_Importances.svg")

plt.show()

#ax.tick_params(axis='y', labelsize=10)
plt.show()